Setup & Advanced Library Installation

In [11]:
# Install Prophet for time series forecasting
!pip install prophet

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv('/content/Sample - Superstore (1).csv', encoding='latin1')
df['Order Date'] = pd.to_datetime(df['Order Date'])

print("Phase 1 Complete: Environment Ready.")

Phase 1 Complete: Environment Ready.


Data Preprocessing (Professional Clean-up)

In [12]:
# Aggregating sales to Monthly level to see clear trends
monthly_sales = df.set_index('Order Date').resample('MS')['Sales'].sum().reset_index()

# Renaming columns for Prophet compatibility
prophet_df = monthly_sales.rename(columns={'Order Date': 'ds', 'Sales': 'y'})

print(f"Data Prepared: {len(monthly_sales)} months of historical data found.")
monthly_sales.head()

Data Prepared: 48 months of historical data found.


,Order Date,Sales
0,2014-01-01,14236.895
1,2014-02-01,4519.892
2,2014-03-01,55691.009
3,2014-04-01,28295.345
4,2014-05-01,23648.287


Training the AI Model (Prophet with Seasonality)

In [13]:
# Initialize Model with confidence interval (95%)
model = Prophet(yearly_seasonality=True, interval_width=0.95)

# Fit the model on our data
model.fit(prophet_df)

# Create a timeline for the next 12 months (future)
future = model.make_future_dataframe(periods=12, freq='MS')

# Predict sales for the future
forecast = model.predict(future)

print("Phase 3 Complete: AI Model Trained and Prediction Generated.")

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Phase 3 Complete: AI Model Trained and Prediction Generated.


Model Evaluation (Internship Rigor)

In [14]:
# Calculate Error Metrics on historical data
# We compare the 'yhat' (prediction) against 'y' (actual)
actual = prophet_df['y']
predicted = forecast.iloc[:len(actual)]['yhat']

mae = mean_absolute_error(actual, predicted)
rmse = np.sqrt(mean_squared_error(actual, predicted))
mape = np.mean(np.abs((actual - predicted) / actual)) * 100

print(f"--- Model Performance ---")
print(f"Mean Absolute Error (MAE): ${mae:.2f}")
print(f"Root Mean Squared Error (RMSE): ${rmse:.2f}")
print(f"Accuracy (MAPE): {100 - mape:.2f}%")

--- Model Performance ---
Mean Absolute Error (MAE): $5203.24
Root Mean Squared Error (RMSE): $6762.71
Accuracy (MAPE): 88.50%


The "Interactive Dashboard" (Plotly)

In [15]:
# Create an interactive dashboard plot
fig = go.Figure()

# 1. Historical Data
fig.add_trace(go.Scatter(x=prophet_df['ds'], y=prophet_df['y'], name='Actual Sales',
                         line=dict(color='royalblue', width=3)))

# 2. Predicted Data (Forecast)
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='AI Forecast',
                         line=dict(color='firebrick', width=3, dash='dash')))

# 3. Confidence Interval (Shadow area)
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill=None, mode='lines',
                         line_color='rgba(0,0,0,0)', showlegend=False))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='lines',
                         line_color='rgba(255, 0, 0, 0.1)', name='Confidence Interval'))

# Layout Styling
fig.update_layout(title='<b>AI-Powered Retail Sales Forecasting Dashboard</b>',
                  xaxis_title='Timeline', yaxis_title='Sales ($)',
                  template='plotly_white', hovermode='x unified')

fig.show()

# Save results for Power BI / Excel report
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].to_csv('final_forecast_report.csv', index=False)